In [1]:
print(f"""
Created on Sat Dec 09:00:12 2021

@author: Katharina Sabrina bursy

Machine Learning - DAT-5303 - BMBAN2
Classification Model Development 
    
""")


Created on Sat Dec 09:00:12 2021

@author: Katharina Sabrina bursy

Machine Learning - DAT-5303 - BMBAN2
Classification Model Development 
    



# Import Libraries

In [2]:
#import time for display of hoe long the code takes at the end
import time 
start_time=time.time()

In [3]:
import pandas            as pd                       # data science essentials
import matplotlib.pyplot as plt                      # data visualization
import seaborn           as sns                      # enhanced data viz
from sklearn.model_selection import train_test_split # train-test split
from sklearn.linear_model import LogisticRegression  # logistic regression
import statsmodels.formula.api as smf                # logistic regression
from sklearn.metrics import confusion_matrix         # confusion matrix
from sklearn.metrics import roc_auc_score            # auc score
from sklearn.neighbors import KNeighborsClassifier   # KNN for classification
from sklearn.neighbors import KNeighborsRegressor    # KNN for regression
from sklearn.preprocessing import StandardScaler     # standard scaler
from sklearn.tree import DecisionTreeClassifier      # classification trees
from sklearn.tree import plot_tree


# new packages
from sklearn.model_selection import RandomizedSearchCV     # hyperparameter tuning
from sklearn.metrics import make_scorer              # customizable scorer
# new tools
from sklearn.ensemble import RandomForestClassifier     # random forest
from sklearn.ensemble import GradientBoostingClassifier # gbm


In [4]:
#read the file and display it
file = "./GOT_character_predictions.xlsx"
GOT= pd.read_excel(io=file)
# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)
GOT.head()

,S.No,name,title,culture,dateOfBirth,mother,father,heir,house,spouse,book1_A_Game_Of_Thrones,book2_A_Clash_Of_Kings,book3_A_Storm_Of_Swords,book4_A_Feast_For_Crows,book5_A_Dance_with_Dragons,isAliveMother,isAliveFather,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,popularity,isAlive
0,1,Viserys II Targaryen,NaN,NaN,NaN,Rhaenyra Targaryen,Daemon Targaryen,Aegon IV Targaryen,NaN,NaN,0,0,0,0,0,1.0,0.0,0.0,NaN,0,0,NaN,11,0.605351,0
1,2,Walder Frey,Lord of the Crossing,Rivermen,208.0,NaN,NaN,NaN,House Frey,Perra Royce,1,1,1,1,1,NaN,NaN,NaN,1.0,1,1,97.0,1,0.896321,1
2,3,Addison Hill,Ser,NaN,NaN,NaN,NaN,NaN,House Swyft,NaN,0,0,0,1,0,NaN,NaN,NaN,NaN,0,1,NaN,0,0.267559,1
3,4,Aemma Arryn,Queen,NaN,82.0,NaN,NaN,NaN,House Arryn,Viserys I Targaryen,0,0,0,0,0,NaN,NaN,NaN,0.0,1,1,23.0,0,0.183946,0
4,5,Sylva Santagar,Greenstone,Dornish,276.0,NaN,NaN,NaN,House Santagar,Eldon Estermont,0,0,0,1,0,NaN,NaN,NaN,1.0,1,1,29.0,0,0.043478,1


In [5]:
#check for missing values in the dataset
GOT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1946 entries, 0 to 1945
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   S.No                        1946 non-null   int64  
 1   name                        1946 non-null   object 
 2   title                       938 non-null    object 
 3   culture                     677 non-null    object 
 4   dateOfBirth                 433 non-null    float64
 5   mother                      21 non-null     object 
 6   father                      26 non-null     object 
 7   heir                        23 non-null     object 
 8   house                       1519 non-null   object 
 9   spouse                      276 non-null    object 
 10  book1_A_Game_Of_Thrones     1946 non-null   int64  
 11  book2_A_Clash_Of_Kings      1946 non-null   int64  
 12  book3_A_Storm_Of_Swords     1946 non-null   int64  
 13  book4_A_Feast_For_Crows     1946 

In [6]:
GOT=GOT.rename(columns={'S.No':'Order'})
#renaming the column since it gave issues putting it with a dot in regression or other
#not that the order has any impact anyway

In [7]:
#Used variable that I did not want in the dataset later on
# GOT['numDeadRelations'].value_counts()
# GOT['D_numDeadRelations']=0
# for index, value in GOT.iterrows():
#     if GOT.loc[index, 'numDeadRelations'] ==0:
#         GOT.loc[index, 'D_numDeadRelations'] = 1
# GOT['D_numDeadRelations'].value_counts()

# Missing Values
Considering the high amount of missing data in the set, we created categories that says
has an unknown value=1
does have a value=0

In [8]:
# instantiating a missing value flag for mother
GOT['mother_unknown'] = GOT.loc[:, 'mother'].isnull().astype(int)
GOT['mother_unknown'].value_counts()

1    1925
0      21
Name: mother_unknown, dtype: int64

In [9]:
# instantiating a missing value flag for father
GOT['father_unknown'] = GOT.loc[:, 'father'].isnull().astype(int)
GOT['father_unknown'].value_counts()

1    1920
0      26
Name: father_unknown, dtype: int64

In [10]:
# instantiating a missing value flag for house
GOT['house_unknown']= GOT.loc[:, 'house'].isnull().astype(int)
GOT['house_unknown'].value_counts()

0    1519
1     427
Name: house_unknown, dtype: int64

In [11]:
# instantiating a missing value flag for culture
GOT['culture_unknown']= GOT.loc[:, 'culture'].isnull().astype(int)
GOT['culture_unknown'].value_counts()

1    1269
0     677
Name: culture_unknown, dtype: int64

group the top 10 and use the others as others
use gender guesser for some values,might give some more

In [12]:
# instantiating a missing value flag for title
GOT['title_unknown']= GOT.loc[:, 'title'].isnull().astype(int)
GOT['title_unknown'].value_counts()

1    1008
0     938
Name: title_unknown, dtype: int64

In [13]:
# instantiating a missing value flag for heir
GOT['heir_unknown']= GOT.loc[:, 'heir'].isnull().astype(int)
GOT['heir_unknown'].value_counts() #less uselful because not big enough

1    1923
0      23
Name: heir_unknown, dtype: int64

In [14]:
# instantiating a missing value flag for spouse
GOT['spouse_unknown'] = GOT.loc[:, 'spouse'].isnull().astype(int)
GOT['spouse_unknown'].value_counts()

1    1670
0     276
Name: spouse_unknown, dtype: int64

In [15]:
# instantiating a missing value flag for age
GOT['age_unknown'] = GOT.loc[:, 'age'].isnull().astype(int)
GOT['age_unknown'].value_counts()

1    1513
0     433
Name: age_unknown, dtype: int64

In [16]:
# instantiating a missing value flag for mother alive
GOT['isAliveMother_unknown'] = GOT.loc[:, 'isAliveMother'].isnull().astype(int)
GOT['isAliveMother_unknown'].value_counts()

1    1925
0      21
Name: isAliveMother_unknown, dtype: int64

In [17]:
# instantiating a missing value flag for father alive
GOT['isAliveFather_unknown'] = GOT.loc[:, 'isAliveFather'].isnull().astype(int)
GOT['isAliveFather_unknown'].value_counts()

1    1920
0      26
Name: isAliveFather_unknown, dtype: int64

In [18]:
# instantiating a missing value flag for heir alive
GOT['isAliveHeir_unknown'] = GOT.loc[:, 'isAliveHeir'].isnull().astype(int)
GOT['isAliveHeir_unknown'].value_counts()

1    1923
0      23
Name: isAliveHeir_unknown, dtype: int64

In [19]:
# instantiating a missing value flag for spouse alive
GOT['isAliveSpouse_unknown'] = GOT.loc[:, 'isAliveSpouse'].isnull().astype(int)
GOT['isAliveSpouse_unknown'].value_counts()

1    1670
0     276
Name: isAliveSpouse_unknown, dtype: int64

In [20]:
# instantiating a missing value flag for parent alive
GOT['isAliveParent_unknown'] = GOT.loc[:, 'isAliveMother'].isnull().astype(int)+GOT.loc[:, 'isAliveFather'].isnull().astype(int)
GOT['isAliveParent_unknown'].value_counts()

2    1920
0      21
1       5
Name: isAliveParent_unknown, dtype: int64

In [21]:
#check if it makes sense that the whole family is alive or not
#if one does have an 
GOT['isAliveFamily_unknown']=0
for index, value in GOT.iterrows():
    if GOT.loc[index, 'isAliveMother_unknown'] ==0 or GOT.loc[index, 'isAliveFather_unknown'] ==0\
    or GOT.loc[index, 'isAliveSpouse_unknown'] ==0:
        GOT.loc[index, 'isAliveFamily_unknown'] = 1
GOT['isAliveFamily_unknown'].value_counts()

0    1644
1     302
Name: isAliveFamily_unknown, dtype: int64

In [22]:
GOT['familiarity_mother']=0
for index, value in GOT.iterrows():
    if GOT.loc[index, 'mother_unknown'] ==0 or GOT.loc[index, 'culture_unknown'] ==0:
        GOT.loc[index, 'familiarity_mother'] = 1
GOT['familiarity_mother'].value_counts()

0    1250
1     696
Name: familiarity_mother, dtype: int64

In [23]:
GOT['familiarity_mother_2']=0
for index, value in GOT.iterrows():
    if GOT.loc[index, 'isAliveMother'] ==0 or GOT.loc[index, 'culture_unknown'] ==0:
        GOT.loc[index, 'familiarity_mother_2'] = 1
GOT['familiarity_mother_2'].value_counts()

0    1263
1     683
Name: familiarity_mother_2, dtype: int64

In [24]:
GOT['familiarity_father']=0
for index, value in GOT.iterrows():
    if GOT.loc[index, 'isAliveFather'] ==0 or GOT.loc[index, 'house_unknown'] ==0:
        GOT.loc[index, 'familiarity_father'] = 1
GOT['familiarity_father'].value_counts()

1    1540
0     406
Name: familiarity_father, dtype: int64

In [25]:
GOT['familiarity_father_2']=0
for index, value in GOT.iterrows():
    if GOT.loc[index, 'father_unknown'] ==0 or GOT.loc[index, 'house_unknown'] ==0:
        GOT.loc[index, 'familiarity_father_2'] = 1
GOT['familiarity_father_2'].value_counts()

1    1545
0     401
Name: familiarity_father_2, dtype: int64

In [26]:
# instantiating a missing value flag for carat
GOT['nobel_real'] = 0
for index, value in GOT.iterrows():
    if GOT.loc[index, 'title_unknown'] ==0 and GOT.loc[index, 'isNoble'] ==1:
        GOT.loc[index, 'nobel_real'] = 1
GOT['nobel_real'].value_counts()

0    1049
1     897
Name: nobel_real, dtype: int64

In [27]:
GOT['popularity_survive'] = GOT['popularity'] + GOT['numDeadRelations'] 
#GOT['popularity_survive'].value_counts()

In [28]:
GOT['heir_survive'] = 0
for index, value in GOT.iterrows():
    if GOT.loc[index, 'isAliveFather'] ==1 or GOT.loc[index, 'familiarity_mother_2'] ==1:
        GOT.loc[index, 'heir_survive'] = 1
GOT['heir_survive'].value_counts()

0    1259
1     687
Name: heir_survive, dtype: int64

In [29]:
GOT['heir_survive_2'] = GOT['popularity_survive']*GOT['familiarity_father']
GOT['heir_survive_2'].value_counts()

0.000000     457
0.016722      80
0.013378      79
0.010033      70
0.023411      67
0.020067      60
0.026756      58
0.006689      55
0.043478      37
0.033445      37
0.030100      36
0.003344      35
0.050167      29
0.036789      28
0.046823      27
0.040134      25
0.063545      23
0.060201      23
0.070234      21
0.053512      20
0.073579      19
0.076923      18
0.066890      18
0.086957      18
0.080268      16
0.056856      16
0.140468      14
0.130435      13
0.083612      13
0.127090      11
0.090301      11
0.103679      11
0.137124      11
0.133779      10
0.013378      10
0.107023      10
0.160535       8
0.093645       8
0.143813       8
0.020067       8
0.016722       8
0.100334       7
0.006689       7
0.120401       7
0.147157       6
0.123746       6
0.026756       6
0.003344       6
0.163880       6
0.046823       6
0.110368       6
0.153846       6
0.167224       5
0.113712       5
0.096990       5
0.043478       4
0.086957       4
0.187291       4
0.096990      

In [30]:
GOT['heir_survive_3'] = GOT['popularity_survive']+GOT['familiarity_father']
GOT['heir_survive_3'].value_counts()

1.016722     80
1.013378     79
1.010033     70
1.023411     67
1.020067     60
1.026756     58
1.006689     55
1.000000     51
0.006689     48
0.013378     41
0.003344     41
1.033445     37
1.043478     37
1.030100     36
1.003344     35
0.016722     33
0.000000     33
0.010033     31
1.050167     29
1.036789     28
1.046823     27
1.040134     25
1.063545     23
1.060201     23
1.070234     21
1.053512     20
1.073579     19
1.066890     18
0.020067     18
1.076923     18
1.086957     18
1.080268     16
1.056856     16
0.023411     14
1.140468     14
1.130435     14
1.083612     13
1.090301     11
1.127090     11
1.103679     11
1.137124     11
1.013378     10
0.026756     10
1.133779     10
1.107023     10
0.040134      9
1.020067      8
1.093645      8
1.143813      8
1.016722      8
0.030100      8
1.160535      8
1.006689      7
1.120401      7
1.100334      7
1.026756      6
1.046823      6
1.003344      6
0.036789      6
1.110368      6
1.153846      6
0.043478      6
0.050167

In [31]:
GOT['alive_fam']=0
for index, value in GOT.iterrows():
    if GOT.loc[index, 'isAliveFather'] ==1 or GOT.loc[index, 'isAliveMother'] ==1 or GOT.loc[index, 'isAliveMother'] :
        GOT.loc[index, 'heir_survive'] = 1
GOT['heir_survive'].value_counts()

1    1946
Name: heir_survive, dtype: int64

In [32]:
GOT['alive_books']= GOT['book1_A_Game_Of_Thrones']+GOT['book2_A_Clash_Of_Kings']+ GOT['book3_A_Storm_Of_Swords']\
+GOT['book4_A_Feast_For_Crows']+GOT['book5_A_Dance_with_Dragons']
GOT['alive_books'].value_counts()

1    663
2    331
3    286
0    272
5    212
4    182
Name: alive_books, dtype: int64

In [33]:
GOT['alive_books_longest']=0

In [34]:
for index, value in GOT.iterrows():
    if GOT.loc[index, 'alive_books'] ==5 :
        GOT.loc[index, 'alive_books_longest'] = 1

In [35]:
GOT['alive_books_longest'].value_counts()

0    1734
1     212
Name: alive_books_longest, dtype: int64

# Gender guesser

Commented out for time reasons and hardcoded at the bottom

In [36]:
# import random as rand # random number generation
# # New! (may need to be downloaded)
# import gender_guesser.detector as gender # guess gender based on (given) name
# # setting random seed
# rand.seed(a = 219)

In [37]:
# # STEP 1: splitting names 

# # placeholder list
# placeholder_lst = []

# # looping over each email address
# for index, col in GOT.iterrows():
    
#     # splitting name by the ' ' in the middle
#     split_name = GOT.loc[index, 'name'].split(sep = ' ')
    
#     # appending placeholder_lst with the results
#     placeholder_lst.append(split_name)
    

# # converting placeholder_lst into a DataFrame 
# name_df = pd.DataFrame(placeholder_lst)


# # displaying the results
# name_df

In [38]:
# #gender guesser needs the first column with the first name
# #therefore we can drop the other columns
# name_df=name_df.drop(5,axis=1)
# name_df
# name_df=name_df.drop(4,axis=1)
# name_df
# name_df=name_df.drop(3,axis=1)
# name_df
# name_df=name_df.drop(2,axis=1)
# name_df
# name_df=name_df.drop(1,axis=1)
# name_df

In [39]:
# # renaming column for easier use
# name_df = name_df.rename(columns={name_df.columns[0]: 'First_name'})
# name_df

In [40]:
# GOT = pd.concat([GOT, name_df['First_name']],
#                    axis = 1)

In [41]:
# # guessing gender based on (given) name

# # placeholder list
# placeholder_lst=[]


# # looping to guess gender
# for name in GOT['First_name']:
#     guess = gender.Detector().get_gender(name)
#     print(guess)
#     placeholder_lst.append(guess)


# # converting list into a series
# GOT['gender_guess'] = pd.Series(placeholder_lst)


# # checking results
# GOT

In [42]:
# for val in GOT['gender_guess']:
#     print(f"'{val}',")

In [43]:
gender_guesser = ['unknown',
'unknown',
'andy',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'male',
'male',
'mostly_male',
'mostly_male',
'mostly_male',
'mostly_male',
'mostly_male',
'mostly_male',
'unknown',
'male',
'unknown',
'unknown',
'male',
'male',
'female',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'mostly_female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'male',
'male',
'andy',
'andy',
'unknown',
'andy',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'male',
'male',
'unknown',
'male',
'male',
'male',
'male',
'male',
'male',
'male',
'mostly_male',
'male',
'mostly_male',
'mostly_male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'mostly_male',
'unknown',
'unknown',
'male',
'female',
'andy',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'mostly_male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'mostly_female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'andy',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'male',
'female',
'female',
'female',
'female',
'female',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'mostly_female',
'female',
'unknown',
'mostly_female',
'unknown',
'female',
'unknown',
'female',
'unknown',
'male',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'andy',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'male',
'male',
'male',
'male',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'female',
'female',
'female',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'mostly_female',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'male',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'mostly_male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'mostly_male',
'female',
'male',
'male',
'male',
'female',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'male',
'male',
'male',
'female',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'andy',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'female',
'male',
'unknown',
'unknown',
'female',
'male',
'unknown',
'male',
'unknown',
'unknown',
'male',
'female',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'female',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'male',
'unknown',
'male',
'male',
'unknown',
'unknown',
'male',
'male',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'female',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'female',
'female',
'female',
'unknown',
'unknown',
'male',
'male',
'male',
'male',
'male',
'male',
'male',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'mostly_male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'mostly_female',
'mostly_female',
'mostly_female',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'female',
'male',
'male',
'male',
'male',
'unknown',
'female',
'female',
'female',
'unknown',
'mostly_male',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'female',
'male',
'female',
'unknown',
'unknown',
'female',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'unknown',
'female',
'unknown',
'female',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'female',
'unknown',
'male',
'unknown',
'unknown',
'mostly_female',
'male',
'female',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'female',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'male',
'male',
'female',
'mostly_female',
'female',
'mostly_female',
'mostly_female',
'mostly_female',
'mostly_female',
'mostly_female',
'mostly_female',
'unknown',
'unknown',
'female',
'female',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'female',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'female',
'unknown',
'female',
'unknown',
'unknown',
'female',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'female',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'female',
'unknown',
'unknown',
'male',
'male',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'female',
'mostly_male',
'unknown',
'female',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'male',
'male',
'male',
'male',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'male',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'male',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'male',
'male',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'female',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'female',
'unknown',
'female',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'mostly_male',
'male',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'female',
'female',
'female',
'male',
'male',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'male',
'unknown',
'female',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'male',
'male',
'andy',
'male',
'male',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'female',
'unknown',
'male',
'male',
'male',
'male',
'male',
'male',
'mostly_male',
'mostly_male',
'mostly_male',
'mostly_male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'male',
'unknown',
'male',
'male',
'unknown',
'male',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'female',
'female',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'male',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'female',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'mostly_male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'mostly_female',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'mostly_female',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'male',
'mostly_female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'female',
'unknown',
'female',
'unknown',
'female',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'andy',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'female',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'mostly_female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'male',
'unknown',
'male',
'male',
'unknown',
'male',
'unknown',
'male',
'male',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'male',
'male',
'male',
'male',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'mostly_male',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'mostly_female',
'unknown',
'unknown',
'unknown',
'female',
'male',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'female',
'male',
'mostly_male',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'female',
'unknown',
'unknown',
'unknown',
'male',
'unknown',
'unknown']

In [44]:
#match the hardcoded list to the dataframe
GOT['gender_guess']=gender_guesser

In [45]:
#check the values and if each categoryhas enough
GOT['gender_guess'].value_counts()

unknown          1385
male              381
female            125
mostly_male        24
mostly_female      21
andy               10
Name: gender_guess, dtype: int64

## M_gender & is_female
if the gender is unknown flag it othewise 0




do this to avoid that some categories have a lov amount of values

In [46]:
#simple loop to create the new variable 
GOT['m_gender']=0
for index, value in GOT.iterrows():
    if GOT.loc[index, 'gender_guess'] == 'unknown' :
        GOT.loc[index, 'm_gender'] = 1
GOT['m_gender'].value_counts()

1    1385
0     561
Name: m_gender, dtype: int64

In [47]:
#simple loop to create the new variable 
#if the name is mostly female or female give it a one otherwise 0
GOT['is_female']=0
for index, value in GOT.iterrows():
    if GOT.loc[index, 'gender_guess'] == 'female' or GOT.loc[index, 'gender_guess'] == 'mostly_female' :
        GOT.loc[index, 'is_female'] = 1
GOT['is_female'].value_counts()

0    1800
1     146
Name: is_female, dtype: int64

# Optimal neighbors

In [48]:
########################################
# optimal_neighbors
########################################
def optimal_neighbors(x_data,
                      y_data,
                      standardize = True,
                      pct_test=0.10,
                      seed=219,
                      response_type='reg',
                      max_neighbors=20,
                      show_viz=True):
    """
Exhaustively compute training and testing results for KNN across
[1, max_neighbors]. Outputs the maximum test score and (by default) a
visualization of the results.
PARAMETERS
----------
x_data        : explanatory variable data
y_data        : response variable
standardize   : whether or not to standardize the x data, default True
pct_test      : test size for training and validation from (0,1), default 0.25
seed          : random seed to be used in algorithm, default 219
response_type : type of neighbors algorithm to use, default 'reg'
    Use 'reg' for regression (KNeighborsRegressor)
    Use 'class' for classification (KNeighborsClassifier)
max_neighbors : maximum number of neighbors in exhaustive search, default 20
show_viz      : display or surpress k-neigbors visualization, default True
"""    
    
    
    if standardize == True:
        # optionally standardizing x_data
        scaler             = StandardScaler()
        scaler.fit(x_data)
        x_scaled           = scaler.transform(x_data)
        x_scaled_df        = pd.DataFrame(x_scaled)
        x_data             = x_scaled_df



    # train-test split
    x_train, x_test, y_train, y_test = train_test_split(x_data,
                                                        y_data,
                                                        test_size = pct_test,
                                                        random_state = seed)


    # creating lists for training set accuracy and test set accuracy
    training_accuracy = []
    test_accuracy = []
    
    
    # setting neighbor range
    neighbors_settings = range(1, max_neighbors + 1)


    for n_neighbors in neighbors_settings:
        # building the model based on response variable type
        if response_type == 'reg':
            clf = KNeighborsRegressor(n_neighbors = n_neighbors)
            clf.fit(x_train, y_train)
            
        elif response_type == 'class':
            clf = KNeighborsClassifier(n_neighbors = n_neighbors)
            clf.fit(x_train, y_train)            
            
        else:
            print("Error: response_type must be 'reg' or 'class'")
        
        
        # recording the training set accuracy
        training_accuracy.append(clf.score(x_train, y_train))
    
        # recording the generalization accuracy
        test_accuracy.append(clf.score(x_test, y_test))


    # optionally displaying visualization
    if show_viz == True:
        # plotting the visualization
        fig, ax = plt.subplots(figsize=(12,8))
        plt.plot(neighbors_settings, training_accuracy, label = "training accuracy")
        plt.plot(neighbors_settings, test_accuracy, label = "test accuracy")
        plt.ylabel("Accuracy")
        plt.xlabel("n_neighbors")
        plt.legend()
        plt.show()
    
    
    # returning optimal number of neighbors
    print(f"The optimal number of neighbors is: {test_accuracy.index(max(test_accuracy))+1}")
    return test_accuracy.index(max(test_accuracy))+1


########################################
# visual_cm
########################################
def visual_cm(true_y, pred_y, labels = None):
    """
Creates a visualization of a confusion matrix.

PARAMETERS
----------
true_y : true values for the response variable
pred_y : predicted values for the response variable
labels : , default None
    """
    # visualizing the confusion matrix

    # setting labels
    lbls = labels
    

    # declaring a confusion matrix object
    cm = confusion_matrix(y_true = true_y,
                          y_pred = pred_y)


    # heatmap
    sns.heatmap(cm,
                annot       = True,
                xticklabels = lbls,
                yticklabels = lbls,
                cmap        = 'Blues',
                fmt         = 'g')


    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix of the Classifier')
    plt.show()

In [49]:
#Initally check how the given variables are correlated to the y-variable
df_corr = GOT.corr(method='pearson').round(2)

df_corr['isAlive'].sort_values(ascending = False)

isAlive                       1.00
isAliveHeir                   0.38
book4_A_Feast_For_Crows       0.27
isAliveFather                 0.20
isAliveSpouse                 0.17
age_unknown                   0.15
mother_unknown                0.14
isAliveMother_unknown         0.14
father_unknown                0.14
isAliveFather_unknown         0.14
isAliveParent_unknown         0.14
isAliveHeir_unknown           0.13
heir_unknown                  0.13
age                           0.09
isAliveSpouse_unknown         0.05
spouse_unknown                0.05
is_female                     0.05
culture_unknown               0.04
alive_books                   0.04
title_unknown                 0.04
house_unknown                 0.04
book5_A_Dance_with_Dragons    0.03
book3_A_Storm_Of_Swords       0.01
alive_books_longest          -0.01
m_gender                     -0.03
isNoble                      -0.04
isAliveMother                -0.04
nobel_real                   -0.04
familiarity_mother_2

In [50]:
#chekc the y- variabe and how the value_counts are
GOT.loc[ : ,'isAlive'].value_counts(normalize = True).round(decimals = 2)

1    0.75
0    0.25
Name: isAlive, dtype: float64

## Drop unnecessary variables with missing values

In [51]:
#create a copy of the dataset, just in case
GOT_data_2= GOT.copy()
GOT_data_2

,Order,name,title,culture,dateOfBirth,mother,father,heir,house,spouse,book1_A_Game_Of_Thrones,book2_A_Clash_Of_Kings,book3_A_Storm_Of_Swords,book4_A_Feast_For_Crows,book5_A_Dance_with_Dragons,isAliveMother,isAliveFather,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,popularity,isAlive,mother_unknown,father_unknown,house_unknown,culture_unknown,title_unknown,heir_unknown,spouse_unknown,age_unknown,isAliveMother_unknown,isAliveFather_unknown,isAliveHeir_unknown,isAliveSpouse_unknown,isAliveParent_unknown,isAliveFamily_unknown,familiarity_mother,familiarity_mother_2,familiarity_father,familiarity_father_2,nobel_real,popularity_survive,heir_survive,heir_survive_2,heir_survive_3,alive_fam,alive_books,alive_books_longest,gender_guess,m_gender,is_female
0,1,Viserys II Targaryen,NaN,NaN,NaN,Rhaenyra Targaryen,Daemon Targaryen,Aegon IV Targaryen,NaN,NaN,0,0,0,0,0,1.0,0.0,0.0,NaN,0,0,NaN,11,0.605351,0,0,0,1,1,1,0,1,1,0,0,0,1,0,1,1,0,1,1,0,11.605351,1,11.605351,12.605351,0,0,0,unknown,1,0
1,2,Walder Frey,Lord of the Crossing,Rivermen,208.0,NaN,NaN,NaN,House Frey,Perra Royce,1,1,1,1,1,NaN,NaN,NaN,1.0,1,1,97.0,1,0.896321,1,1,1,0,0,0,1,0,0,1,1,1,0,2,1,1,1,1,1,1,1.896321,1,1.896321,2.896321,0,5,1,unknown,1,0
2,3,Addison Hill,Ser,NaN,NaN,NaN,NaN,NaN,House Swyft,NaN,0,0,0,1,0,NaN,NaN,NaN,NaN,0,1,NaN,0,0.267559,1,1,1,0,1,0,1,1,1,1,1,1,1,2,0,0,0,1,1,1,0.267559,1,0.267559,1.267559,0,1,0,andy,0,0
3,4,Aemma Arryn,Queen,NaN,82.0,NaN,NaN,NaN,House Arryn,Viserys I Targaryen,0,0,0,0,0,NaN,NaN,NaN,0.0,1,1,23.0,0,0.183946,0,1,1,0,1,0,1,0,0,1,1,1,0,2,1,0,0,1,1,1,0.183946,1,0.183946,1.183946,0,0,0,unknown,1,0
4,5,Sylva Santagar,Greenstone,Dornish,276.0,NaN,NaN,NaN,House Santagar,Eldon Estermont,0,0,0,1,0,NaN,NaN,NaN,1.0,1,1,29.0,0,0.043478,1,1,1,0,0,0,1,0,0,1,1,1,0,2,1,1,1,1,1,1,0.043478,1,0.043478,1.043478,0,1,0,female,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,1942,Luwin,NaN,Westeros,NaN,NaN,NaN,NaN,House Stark,NaN,1,1,1,1,1,NaN,NaN,NaN,NaN,0,0,NaN,0,0.351171,0,1,1,0,0,1,1,1,1,1,1,1,1,2,0,1,1,1,1,0,0.351171,1,0.351171,1.351171,0,5,1,unknown,1,0
1942,1943,Reek,NaN,NaN,NaN,NaN,NaN,NaN,House Bolton,NaN,0,1,0,1,1,NaN,NaN,NaN,NaN,0,0,NaN,0,0.096990,0,1,1,0,1,1,1,1,1,1,1,1,1,2,0,0,0,1,1,0,0.096990,1,0.096990,1.096990,0,3,0,unknown,1,0
1943,1944,Symeon Star-Eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,NaN,NaN,NaN,NaN,0,0,NaN,0,0.030100,1,1,1,1,1,1,1,1,1,1,1,1,1,2,0,0,0,0,0,0,0.030100,1,0.000000,0.030100,0,5,1,male,0,0
1944,1945,Coldhands,NaN,NaN,NaN,NaN,NaN,NaN,Three-eyed crow,NaN,0,0,1,1,1,NaN,NaN,NaN,NaN,0,0,NaN,0,0.130435,1,1,1,0,1,1,1,1,1,1,1,1,1,2,0,0,0,1,1,0,0.130435,1,0.130435,1.130435,0,3,0,unknown,1,0


In [52]:
GOT_data_2=GOT_data_2.drop(labels='name',axis=1)
GOT_data_2=GOT_data_2.drop(labels='title',axis=1)
GOT_data_2=GOT_data_2.drop(labels='culture',axis=1)
GOT_data_2=GOT_data_2.drop(labels='dateOfBirth',axis=1)
GOT_data_2=GOT_data_2.drop(labels='mother',axis=1)
GOT_data_2=GOT_data_2.drop(labels='father',axis=1)
GOT_data_2=GOT_data_2.drop(labels='heir',axis=1)
GOT_data_2=GOT_data_2.drop(labels='house',axis=1)
GOT_data_2=GOT_data_2.drop(labels='spouse',axis=1)
GOT_data_2=GOT_data_2.drop(labels='isAliveMother',axis=1)
GOT_data_2=GOT_data_2.drop(labels='isAliveFather',axis=1)
GOT_data_2=GOT_data_2.drop(labels='isAliveSpouse',axis=1)
GOT_data_2=GOT_data_2.drop(labels='isAliveHeir',axis=1)
GOT_data_2=GOT_data_2.drop(labels='age',axis=1)
#drop all the variables that are later not needed for the full dataset
#not needed since new features created and these contain the missing values 

In [53]:
# declaring explanatory variables
GOT_data_2=GOT.drop('isAlive', axis=1)


# declaring response variable
GOT_target= GOT.loc[:,'isAlive']

In [54]:
# train-test split with stratification
x_train, x_test, y_train, y_test = train_test_split(
            GOT_data_2,
            GOT_target,
            test_size    = 0.10,
            random_state = 219,
            stratify     = GOT_target) # preserving balance


# merging training data for statsmodels
GOT_train = pd.concat([x_train, y_train], axis = 1)

In [55]:
print(f"""

Response Variable Proportions (Training Set)
--------------------------------------------
{y_train.value_counts(normalize = True).round(decimals = 2)}



Response Variable Proportions (Testing Set)
--------------------------------------------
{y_test.value_counts(normalize = True).round(decimals = 2)}
""")





Response Variable Proportions (Training Set)
--------------------------------------------
1    0.75
0    0.25
Name: isAlive, dtype: float64



Response Variable Proportions (Testing Set)
--------------------------------------------
1    0.74
0    0.26
Name: isAlive, dtype: float64



## Logistic regressions

In [56]:
#print out all th variables with a plus so thay can asier be inputted into the logistic regressions
for val in GOT_data_2:
    print(f" {val} + ")

 Order + 
 name + 
 title + 
 culture + 
 dateOfBirth + 
 mother + 
 father + 
 heir + 
 house + 
 spouse + 
 book1_A_Game_Of_Thrones + 
 book2_A_Clash_Of_Kings + 
 book3_A_Storm_Of_Swords + 
 book4_A_Feast_For_Crows + 
 book5_A_Dance_with_Dragons + 
 isAliveMother + 
 isAliveFather + 
 isAliveHeir + 
 isAliveSpouse + 
 isMarried + 
 isNoble + 
 age + 
 numDeadRelations + 
 popularity + 
 mother_unknown + 
 father_unknown + 
 house_unknown + 
 culture_unknown + 
 title_unknown + 
 heir_unknown + 
 spouse_unknown + 
 age_unknown + 
 isAliveMother_unknown + 
 isAliveFather_unknown + 
 isAliveHeir_unknown + 
 isAliveSpouse_unknown + 
 isAliveParent_unknown + 
 isAliveFamily_unknown + 
 familiarity_mother + 
 familiarity_mother_2 + 
 familiarity_father + 
 familiarity_father_2 + 
 nobel_real + 
 popularity_survive + 
 heir_survive + 
 heir_survive_2 + 
 heir_survive_3 + 
 alive_fam + 
 alive_books + 
 alive_books_longest + 
 gender_guess + 
 m_gender + 
 is_female + 


### Logit_full 0.478693
Regression with all* the variables



*doesn't include all since otherwise error would occur

In [57]:
# instantiating a logistic regression model object
logistic_small = smf.logit(formula   = """isAlive~ 
Order +  
 book1_A_Game_Of_Thrones + 
 book2_A_Clash_Of_Kings + 
 book3_A_Storm_Of_Swords + 
 book4_A_Feast_For_Crows + 
 culture_unknown + 
 isNoble + 
 numDeadRelations + 
 popularity +
 mother_unknown + 
 father_unknown + 
 house_unknown + 
 title_unknown + 
 heir_unknown + 
 spouse_unknown + 
 age_unknown +
m_gender+ 
 is_female+
  alive_books + 
 alive_books_longest  
 """,
                           data = GOT_train)
 

# FITTING the model object
results_logistic = logistic_small.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.478693
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
=======================================================================
Model:                Logit              Pseudo R-squared:   0.156     
Dependent Variable:   isAlive            AIC:                1718.3836 
Date:                 2021-12-05 16:32   BIC:                1833.2104 
No. Observations:     1751               Log-Likelihood:     -838.19   
Df Model:             20                 LL-Null:            -992.53   
Df Residuals:         1730               LLR p-value:        1.3569e-53
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       6.0000                                           
-----------------------------------------------------------------------
                         Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
Intercept               -2.0521   1.0300 -1.9923 0.0463 -4.0710 -0.0333
Order                   -0.0007   0.0001 -5.1190 0.0000 -0.0009 -0.0004
book1_A_Game_Of_Thrones -0.8696   0.2384 -3.6479 0.0003 -1.3368 -0.4024
book2_A_Clash_Of_Kings  -0.3019   0.2101 -1.4366 0.1508 -0.7138  0.1100
book3_A_Storm_Of_Swords -0.2542   0.2171 -1.1711 0.2416 -0.6796  0.1712
book4_A_Feast_For_Crows  1.7434   0.2496  6.9840 0.0000  1.2541  2.2326
culture_unknown          0.0301   0.1369  0.2200 0.8259 -0.2382  0.2985
isNoble                 -0.1505   0.4740 -0.3175 0.7509 -1.0795  0.7786
numDeadRelations        -0.1017   0.0509 -1.9981 0.0457 -0.2015 -0.0019
popularity              -0.3551   0.5375 -0.6605 0.5089 -1.4086  0.6985
mother_unknown           1.8167   1.2331  1.4733 0.1407 -0.6001  4.2334
father_unknown          -0.0320   1.0529 -0.0304 0.9757 -2.0957  2.0316
house_unknown            0.2397   0.1686  1.4210 0.1553 -0.0909  0.5702
title_unknown           -0.0804   0.4707 -0.1708 0.8644 -1.0030  0.8422
heir_unknown             0.7084   0.9115  0.7772 0.4370 -1.0780  2.4949
spouse_unknown           0.1225   0.1878  0.6521 0.5144 -0.2457  0.4906
age_unknown              0.8156   0.1585  5.1444 0.0000  0.5049  1.1263
m_gender                 0.1560   0.1535  1.0167 0.3093 -0.1448  0.4568
is_female                0.4862   0.2813  1.7283 0.0839 -0.0652  1.0376
alive_books              0.0062   0.1566  0.0398 0.9682 -0.3007  0.3132
alive_books_longest      0.9934   0.3116  3.1882 0.0014  0.3827  1.6042
=======================================================================

"""

### Logit 5 0.490102

In [58]:
# instantiating a logistic regression model object
logistic_small = smf.logit(formula   = """isAlive~ 
Order + 
 book1_A_Game_Of_Thrones + 
 book2_A_Clash_Of_Kings + 

 book4_A_Feast_For_Crows + 

 numDeadRelations + 
 popularity+
 alive_books_longest
 

 """,
                           data = GOT_train)
 

# FITTING the model object
results_logistic = logistic_small.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.490102
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
=======================================================================
Model:                Logit              Pseudo R-squared:   0.135     
Dependent Variable:   isAlive            AIC:                1732.3366 
Date:                 2021-12-05 16:32   BIC:                1776.0801 
No. Observations:     1751               Log-Likelihood:     -858.17   
Df Model:             7                  LL-Null:            -992.53   
Df Residuals:         1743               LLR p-value:        2.8375e-54
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       6.0000                                           
-----------------------------------------------------------------------
                         Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
Intercept                1.3979   0.1286 10.8739 0.0000  1.1459  1.6498
Order                   -0.0007   0.0001 -6.1358 0.0000 -0.0010 -0.0005
book1_A_Game_Of_Thrones -0.7259   0.1895 -3.8314 0.0001 -1.0972 -0.3546
book2_A_Clash_Of_Kings  -0.3919   0.1374 -2.8529 0.0043 -0.6611 -0.1227
book4_A_Feast_For_Crows  1.5795   0.1408 11.2183 0.0000  1.3036  1.8555
numDeadRelations        -0.1618   0.0508 -3.1847 0.0014 -0.2613 -0.0622
popularity              -1.4702   0.4600 -3.1959 0.0014 -2.3718 -0.5685
alive_books_longest      0.9715   0.2933  3.3127 0.0009  0.3967  1.5463
=======================================================================

"""

### logit_full_reduced

In [59]:
# instantiating a logistic regression model object
logistic_small = smf.logit(formula   = """isAlive~ 
Order +  
 book1_A_Game_Of_Thrones + 
 book4_A_Feast_For_Crows + 
 mother_unknown + 
 age_unknown 
  

 """,
                           data = GOT_train)
 

# FITTING the model object
results_logistic = logistic_small.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.487631
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
=======================================================================
Model:                Logit              Pseudo R-squared:   0.140     
Dependent Variable:   isAlive            AIC:                1719.6847 
Date:                 2021-12-05 16:32   BIC:                1752.4924 
No. Observations:     1751               Log-Likelihood:     -853.84   
Df Model:             5                  LL-Null:            -992.53   
Df Residuals:         1745               LLR p-value:        7.2629e-58
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       6.0000                                           
-----------------------------------------------------------------------
                         Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
Intercept               -2.3518   0.6487 -3.6251 0.0003 -3.6233 -1.0802
Order                   -0.0008   0.0001 -6.5398 0.0000 -0.0010 -0.0005
book1_A_Game_Of_Thrones -0.6352   0.1449 -4.3843 0.0000 -0.9192 -0.3512
book4_A_Feast_For_Crows  1.7227   0.1382 12.4634 0.0000  1.4518  1.9936
mother_unknown           2.7961   0.6387  4.3779 0.0000  1.5443  4.0479
age_unknown              0.9275   0.1422  6.5246 0.0000  0.6489  1.2062
=======================================================================

"""

### Log_sig 2_2

In [60]:
# instantiating a logistic regression model object
logistic_small = smf.logit(formula   = """isAlive~ 
Order +  
 book1_A_Game_Of_Thrones + 
 book2_A_Clash_Of_Kings + 
 book4_A_Feast_For_Crows +  
 numDeadRelations + 
 mother_unknown + 
 age_unknown

 """,
                           data = GOT_train)
 

# FITTING the model object
results_logistic = logistic_small.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.484654
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
=======================================================================
Model:                Logit              Pseudo R-squared:   0.145     
Dependent Variable:   isAlive            AIC:                1713.2585 
Date:                 2021-12-05 16:32   BIC:                1757.0020 
No. Observations:     1751               Log-Likelihood:     -848.63   
Df Model:             7                  LL-Null:            -992.53   
Df Residuals:         1743               LLR p-value:        2.4217e-58
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       6.0000                                           
-----------------------------------------------------------------------
                         Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
Intercept               -1.6381   0.7008 -2.3376 0.0194 -3.0116 -0.2646
Order                   -0.0007   0.0001 -6.1780 0.0000 -0.0010 -0.0005
book1_A_Game_Of_Thrones -0.4717   0.1553 -3.0375 0.0024 -0.7761 -0.1673
book2_A_Clash_Of_Kings  -0.2724   0.1374 -1.9827 0.0474 -0.5417 -0.0031
book4_A_Feast_For_Crows  1.7668   0.1416 12.4760 0.0000  1.4892  2.0444
numDeadRelations        -0.1114   0.0437 -2.5503 0.0108 -0.1970 -0.0258
mother_unknown           2.2374   0.6790  3.2953 0.0010  0.9066  3.5681
age_unknown              0.7891   0.1498  5.2693 0.0000  0.4956  1.0826
=======================================================================

"""

### Logit_4

In [61]:
# instantiating a logistic regression model object
logistic_small = smf.logit(formula   = """isAlive~ book4_A_Feast_For_Crows
+age_unknown 
 
+father_unknown
 
       
+house_unknown                 
             
+title_unknown

 """,
                           data = GOT_train)


# FITTING the model object
results_logistic = logistic_small.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.506736
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
=======================================================================
Model:                Logit              Pseudo R-squared:   0.106     
Dependent Variable:   isAlive            AIC:                1786.5886 
Date:                 2021-12-05 16:32   BIC:                1819.3963 
No. Observations:     1751               Log-Likelihood:     -887.29   
Df Model:             5                  LL-Null:            -992.53   
Df Residuals:         1745               LLR p-value:        1.6248e-43
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       6.0000                                           
-----------------------------------------------------------------------
                         Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
Intercept               -2.9402   0.5187 -5.6684 0.0000 -3.9569 -1.9236
book4_A_Feast_For_Crows  1.4107   0.1254 11.2504 0.0000  1.1650  1.6565
age_unknown              1.0799   0.1394  7.7469 0.0000  0.8067  1.3531
father_unknown           2.2866   0.4912  4.6553 0.0000  1.3239  3.2494
house_unknown            0.4555   0.1567  2.9075 0.0036  0.1485  0.7626
title_unknown            0.2521   0.1195  2.1094 0.0349  0.0179  0.4864
=======================================================================

"""

## Logit_Sig, 0.48

In [62]:
#logit_sig
# instantiating a logistic regression model object
logistic_small = smf.logit(formula   = """isAlive~ 
Order +  
 book1_A_Game_Of_Thrones + 
 book2_A_Clash_Of_Kings + 

 book4_A_Feast_For_Crows + 
 
 numDeadRelations + 

 mother_unknown + 


 


 age_unknown
 


             
 """,
                           data = GOT_train)


# FITTING the model object
results_logistic = logistic_small.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.484654
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
=======================================================================
Model:                Logit              Pseudo R-squared:   0.145     
Dependent Variable:   isAlive            AIC:                1713.2585 
Date:                 2021-12-05 16:32   BIC:                1757.0020 
No. Observations:     1751               Log-Likelihood:     -848.63   
Df Model:             7                  LL-Null:            -992.53   
Df Residuals:         1743               LLR p-value:        2.4217e-58
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       6.0000                                           
-----------------------------------------------------------------------
                         Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
Intercept               -1.6381   0.7008 -2.3376 0.0194 -3.0116 -0.2646
Order                   -0.0007   0.0001 -6.1780 0.0000 -0.0010 -0.0005
book1_A_Game_Of_Thrones -0.4717   0.1553 -3.0375 0.0024 -0.7761 -0.1673
book2_A_Clash_Of_Kings  -0.2724   0.1374 -1.9827 0.0474 -0.5417 -0.0031
book4_A_Feast_For_Crows  1.7668   0.1416 12.4760 0.0000  1.4892  2.0444
numDeadRelations        -0.1114   0.0437 -2.5503 0.0108 -0.1970 -0.0258
mother_unknown           2.2374   0.6790  3.2953 0.0010  0.9066  3.5681
age_unknown              0.7891   0.1498  5.2693 0.0000  0.4956  1.0826
=======================================================================

"""

## Logit_unsig_3 0.48

In [63]:
#'logit_unsig_3'
# instantiating a logistic regression model object
logistic_small = smf.logit(formula   = """isAlive~ 
Order +  
 book1_A_Game_Of_Thrones + 
 book2_A_Clash_Of_Kings + 
 book4_A_Feast_For_Crows + 
 numDeadRelations + 
 mother_unknown + 
 familiarity_father_2+ 



age_unknown   

 """,
                           data = GOT_train)


# FITTING the model object
results_logistic = logistic_small.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.483721
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
=======================================================================
Model:                Logit              Pseudo R-squared:   0.147     
Dependent Variable:   isAlive            AIC:                1711.9908 
Date:                 2021-12-05 16:32   BIC:                1761.2023 
No. Observations:     1751               Log-Likelihood:     -847.00   
Df Model:             8                  LL-Null:            -992.53   
Df Residuals:         1742               LLR p-value:        3.2595e-58
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       6.0000                                           
-----------------------------------------------------------------------
                         Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
Intercept               -1.3359   0.7204 -1.8545 0.0637 -2.7478  0.0760
Order                   -0.0007   0.0001 -5.8323 0.0000 -0.0009 -0.0005
book1_A_Game_Of_Thrones -0.4713   0.1553 -3.0338 0.0024 -0.7758 -0.1668
book2_A_Clash_Of_Kings  -0.2602   0.1378 -1.8884 0.0590 -0.5302  0.0099
book4_A_Feast_For_Crows  1.7806   0.1420 12.5359 0.0000  1.5022  2.0590
numDeadRelations        -0.1112   0.0436 -2.5497 0.0108 -0.1967 -0.0257
mother_unknown           2.1421   0.6805  3.1480 0.0016  0.8084  3.4758
familiarity_father_2    -0.2907   0.1627 -1.7866 0.0740 -0.6095  0.0282
age_unknown              0.7648   0.1502  5.0932 0.0000  0.4705  1.0591
=======================================================================

"""

## Logit_sig 2 0.48

In [64]:
#logit_sig_2
# instantiating a logistic regression model object
logistic_small = smf.logit(formula   = """isAlive~
book1_A_Game_Of_Thrones + 
book2_A_Clash_Of_Kings + 
book4_A_Feast_For_Crows +   
mother_unknown +  


age_unknown+
popularity_survive+
Order

 """,
                           data = GOT_train)


# FITTING the model object
results_logistic = logistic_small.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.484631
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
=======================================================================
Model:                Logit              Pseudo R-squared:   0.145     
Dependent Variable:   isAlive            AIC:                1713.1782 
Date:                 2021-12-05 16:32   BIC:                1756.9217 
No. Observations:     1751               Log-Likelihood:     -848.59   
Df Model:             7                  LL-Null:            -992.53   
Df Residuals:         1743               LLR p-value:        2.3280e-58
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       6.0000                                           
-----------------------------------------------------------------------
                         Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
Intercept               -1.5757   0.7092 -2.2217 0.0263 -2.9658 -0.1857
book1_A_Game_Of_Thrones -0.4649   0.1558 -2.9839 0.0028 -0.7703 -0.1595
book2_A_Clash_Of_Kings  -0.2697   0.1374 -1.9632 0.0496 -0.5390 -0.0004
book4_A_Feast_For_Crows  1.7689   0.1416 12.4915 0.0000  1.4914  2.0465
mother_unknown           2.1798   0.6858  3.1786 0.0015  0.8357  3.5239
age_unknown              0.7815   0.1505  5.1918 0.0000  0.4865  1.0766
popularity_survive      -0.1067   0.0416 -2.5655 0.0103 -0.1881 -0.0252
Order                   -0.0007   0.0001 -6.1332 0.0000 -0.0010 -0.0005
=======================================================================

"""

# Classification- Logistic Regressions


## Dictionary of all the above LR

In [65]:
# explanatory sets from last session

# creating a dictionary to store candidate models

candidate_dict = {

 # full model
 'logit_full'   : ['Order' , 'book1_A_Game_Of_Thrones' , 'book2_A_Clash_Of_Kings' , \
                   'book3_A_Storm_Of_Swords' , 'book4_A_Feast_For_Crows' , 'book5_A_Dance_with_Dragons' ,\
                   'isMarried' , 'isNoble' , 'numDeadRelations' , 'popularity' , 'mother_unknown' ,\
                   'father_unknown' , 'house_unknown' , 'culture_unknown' , 'title_unknown' , \
                   'heir_unknown' , 'spouse_unknown' , 'age_unknown' , 'isAliveMother_unknown' , \
                   'isAliveFather_unknown' , 'isAliveHeir_unknown' , 'isAliveSpouse_unknown'], 
    
 'logit_full_remove'   : ['Order' , 'book1_A_Game_Of_Thrones' , 'book2_A_Clash_Of_Kings' , \
                  'book3_A_Storm_Of_Swords','book4_A_Feast_For_Crows' , 'book5_A_Dance_with_Dragons' ,\
                    'numDeadRelations' , 'popularity' ,'culture_unknown' ,'isMarried', 'title_unknown' , \
                   'spouse_unknown' , 'age_unknown' , 'isAliveSpouse_unknown'],
    #significant variables only (set 1)
 'logit_full_reduced'    : ['Order' , 'book1_A_Game_Of_Thrones', 'book4_A_Feast_For_Crows',\
                          'mother_unknown','age_unknown','alive_books_longest'],
    
 # significant variables only (set 1)
 'logit_sig'    : ['Order' , 'book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings',\
                   'book4_A_Feast_For_Crows','numDeadRelations', 'mother_unknown','age_unknown'],
    
  
 # significant variables only (set 2)
 'logit_sig_2'  : ['book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings', 'book4_A_Feast_For_Crows',\
                   'mother_unknown', 'age_unknown', 'popularity_survive', 'Order'],

                   
    # mostly significant variables (set 3)
 'logit_unsig_3'  : ['Order', 'book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings',\
                   'book4_A_Feast_For_Crows', 'numDeadRelations', 'mother_unknown',\
                     'familiarity_father_2','age_unknown'],     
    
    # significant variables only (without 'mother_unknown')
     'logit_sig_2_1'  : ['book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings', 'book4_A_Feast_For_Crows',\
                    'age_unknown', 'popularity_survive', 'Order'],
    
    # significant variables only, variation
     'logit_sig_2_2'  : ['Order', 'book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings',\
                    'book4_A_Feast_For_Crows', 'numDeadRelations', 'mother_unknown', 'age_unknown'],
    
    # significant variables only (set 4)
 'logit_sig_4'  : ['book4_A_Feast_For_Crows', 'age_unknown', 'father_unknown',\
                   'house_unknown', 'title_unknown'], 
    
    # significant variables only (set 5)
 'logit_sig_5'  : ['Order', 'book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings',\
                   'book4_A_Feast_For_Crows', 'numDeadRelations','popularity', 'alive_books_longest'],

}

In [66]:
# printing candidate variable sets
print(f"""
/--------------------------\\
|Explanatory Variable Sets |
\\--------------------------/

Full Model:
-----------
{candidate_dict['logit_full']}

Full Model Reduced:
-----------
{candidate_dict['logit_full_reduced']}

First Significant p-value Model:
--------------------------------
{candidate_dict['logit_sig']}

Second Significant p-value Model:
---------------------------------
{candidate_dict['logit_sig_2']}

Third mostly Significant p-value Model:
-----------
{candidate_dict['logit_unsig_3']}

Second_First Significant p-value Model:
--------------------------------
{candidate_dict['logit_sig_2_1']}

Second_Second Significant p-value Model:
---------------------------------
{candidate_dict['logit_sig_2_2']}

Forth Significant p-value Model:
---------------------------------
{candidate_dict['logit_sig_4']}

Fifth Significant p-value Model:
---------------------------------
{candidate_dict['logit_sig_5']}



""")


/--------------------------\
|Explanatory Variable Sets |
\--------------------------/

Full Model:
-----------
['Order', 'book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings', 'book3_A_Storm_Of_Swords', 'book4_A_Feast_For_Crows', 'book5_A_Dance_with_Dragons', 'isMarried', 'isNoble', 'numDeadRelations', 'popularity', 'mother_unknown', 'father_unknown', 'house_unknown', 'culture_unknown', 'title_unknown', 'heir_unknown', 'spouse_unknown', 'age_unknown', 'isAliveMother_unknown', 'isAliveFather_unknown', 'isAliveHeir_unknown', 'isAliveSpouse_unknown']

Full Model Reduced:
-----------
['Order', 'book1_A_Game_Of_Thrones', 'book4_A_Feast_For_Crows', 'mother_unknown', 'age_unknown', 'alive_books_longest']

First Significant p-value Model:
--------------------------------
['Order', 'book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings', 'book4_A_Feast_For_Crows', 'numDeadRelations', 'mother_unknown', 'age_unknown']

Second Significant p-value Model:
---------------------------------
['book1_A_Game

## Track of scores
#Test all the dictionaries with the initial settings

In [67]:
# logit_full
# Training Accuracy: 0.7847
# Testing  Accuracy: 0.8564
# AUC Score: 0.7462

# logit_sig
# Training Accuracy: 0.7778
# Testing  Accuracy: 0.8513
# AUC Score: 0.7362

# logit_sig_2
# C = 1
# Training Accuracy: 0.7813
# Testing  Accuracy: 0.8615
# AUC Score: 0.7497

# ********logit_sig_2
# solver = 'lbfgs',
# C = 2,
# random_state = 219,
# max_iter = 10000
# Training Accuracy: 0.7727
# Testing  Accuracy: 0.8615
# AUC Score: 0.7562

# logit_unsig_3
# Training Accuracy: 0.7807
# Testing  Accuracy: 0.8513
# AUC Score: 0.7362

#log_sig_4
# Training Accuracy: 0.7841
# Testing  Accuracy: 0.8205
# AUC Score: 0.65

### Full dataset

In [68]:
# train/test split with the full model
GOT_data_2   =  GOT.loc[ : , candidate_dict['logit_full']]
GOT_target =  GOT.loc[ : , 'isAlive']


# This is the exact code we were using before
x_train, x_test, y_train, y_test = train_test_split(
            GOT_data_2,
            GOT_target,
            test_size    = 0.10,
            random_state = 219,
            stratify     = GOT_target)


# INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'lbfgs',
                            C = 2.0,
                            random_state = 219,
                            max_iter = 10000)


# FITTING the training data
logreg_fit = logreg.fit(x_train, y_train)

# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(x_test)

# SCORING the results
print('Training Accuracy:', logreg_fit.score(x_train, y_train).round(4))
print('Testing  Accuracy:', logreg_fit.score(x_test, y_test).round(4))

# area under the roc curve (auc)
print('AUC Score:', roc_auc_score(y_true  = y_test,
                    y_score = logreg_pred).round(decimals = 4))


# saving scoring data for future use
logreg_train_score = logreg_fit.score(x_train, y_train).round(4) # accuracy
logreg_test_score  = logreg_fit.score(x_test, y_test).round(4) # accuracy

# saving AUC score for future use
logreg_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = logreg_pred).round(decimals = 4)

Training Accuracy: 0.7853
Testing  Accuracy: 0.8564
AUC Score: 0.7462


### Best variation from dictionary

In [69]:
# train/test split with the full model
GOT_data_2   =  GOT.loc[ : , candidate_dict['logit_sig_2']]
GOT_target =  GOT.loc[ : , 'isAlive']


# This is the exact code we were using before
x_train, x_test, y_train, y_test = train_test_split(
            GOT_data_2,
            GOT_target,
            test_size    = 0.10,
            random_state = 219,
            stratify     = GOT_target)


# INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'lbfgs',
                            C = 2.1,
                            random_state = 219,
                            max_iter = 10000)


# FITTING the training data
logreg_fit = logreg.fit(x_train, y_train)

# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(x_test)

# SCORING the results
print('Training Accuracy:', logreg_fit.score(x_train, y_train).round(4))
print('Testing  Accuracy:', logreg_fit.score(x_test, y_test).round(4))

# area under the roc curve (auc)
print('AUC Score:', roc_auc_score(y_true  = y_test,
                    y_score = logreg_pred).round(decimals = 4))


# saving scoring data for future use
logreg_train_score = logreg_fit.score(x_train, y_train).round(4) # accuracy
logreg_test_score  = logreg_fit.score(x_test, y_test).round(4) # accuracy

# saving AUC score for future use
logreg_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = logreg_pred).round(decimals = 4)

Training Accuracy: 0.7756
Testing  Accuracy: 0.8615
AUC Score: 0.7562


In [70]:
# zipping each feature name to its coefficient
logreg_model_values = zip(GOT[candidate_dict['logit_sig_2']].columns,
                          logreg_fit.coef_.ravel().round(decimals = 2))


# setting up a placeholder list to store model features
logreg_model_lst = [('intercept', logreg_fit.intercept_[0].round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in logreg_model_values:
    logreg_model_lst.append(val)
    

# checking the results
for pair in logreg_model_lst:
    print(pair)

('intercept', 0.32)
('book1_A_Game_Of_Thrones', -0.44)
('book2_A_Clash_Of_Kings', -0.32)
('book4_A_Feast_For_Crows', 1.77)
('mother_unknown', 0.49)
('age_unknown', 0.57)
('popularity_survive', -0.16)
('Order', -0.0)


In [71]:
# # train/test split with the logit_sig variables
# GOT_data_2   =  GOT.loc[:, candidate_dict['logit_sig_2']]
# GOT_target =  GOT.loc[:,'isAlive']


# # train/test split
# x_train, x_test, y_train, y_test = train_test_split(
#             GOT_data_2,
#             GOT_target,
#             random_state = 219,
#             test_size    = 0.1,
#             stratify     = GOT_target)

## Tuning 
For the tuning, I decided to go with values that scikit learn suggested as good ranges

In [72]:
# ########################################
# # RandomizedSearchCV
# ########################################

# # declaring a hyperparameter space
# C_range          = np.arange(0.5, 6.0, 0.1)
# warm_start_range = [True, False]
# solver_range     = ['lbfgs']


# # creating a hyperparameter grid
# param_grid = {'C'          : C_range,
#               'warm_start' : warm_start_range,
#               'solver'     : solver_range}


# # INSTANTIATING the model object without hyperparameters
# lr_tuned = LogisticRegression(random_state = 219,
#                               max_iter     = 3000) # increased for convergence


# # GridSearchCV object
# lr_tuned_cv = RandomizedSearchCV(estimator           = lr_tuned,   # the model object
#                                  param_distributions = param_grid, # parameters to tune
#                                  cv                  = 3,          # how many folds in cross-validation
#                                  n_iter              = 10000,        # number of combinations of hyperparameters to try
#                                  random_state        = 219,        # starting point for random sequence
#                                  scoring = make_scorer(
#                                            roc_auc_score,
#                                            needs_threshold = False)) # scoring criteria (AUC)


# # FITTING to the FULL DATASET (due to cross-validation)
# lr_tuned_cv.fit(GOT_data_2, GOT_target)


# # PREDICT step is not needed


# # printing the optimal parameters and best score
# print("Tuned Parameters  :", lr_tuned_cv.best_params_)
# print("Tuned CV AUC      :", lr_tuned_cv.best_score_.round(4))

In [73]:
# lr_tuned_cv.best_estimator_

### Tuning 1-AUC Score: 0.7562

In [74]:
# train/test split with the full model
GOT_data   =  GOT.loc[ : , candidate_dict['logit_sig_2']]
GOT_target =  GOT.loc[ : , 'isAlive']


# This is the exact code we were using before
x_train, x_test, y_train, y_test = train_test_split(
            GOT_data,
            GOT_target,
            test_size    = 0.10,
            random_state = 219,
            stratify     = GOT_target)


# INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'lbfgs',
                            C = 2,
                            warm_start=True,
                            random_state = 219,
                            max_iter = 3000)


# FITTING the training data
logreg_fit = logreg.fit(x_train, y_train)

# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(x_test)

# SCORING the results
print('Training Accuracy:', logreg_fit.score(x_train, y_train).round(4))
print('Testing  Accuracy:', logreg_fit.score(x_test, y_test).round(4))

# area under the roc curve (auc)
print('AUC Score:', roc_auc_score(y_true  = y_test,
                    y_score = logreg_pred).round(decimals = 4))


# saving scoring data for future use
logreg_train_score = logreg_fit.score(x_train, y_train).round(4) # accuracy
logreg_test_score  = logreg_fit.score(x_test, y_test).round(4) # accuracy

# saving AUC score for future use
logreg_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = logreg_pred).round(decimals = 4)

Training Accuracy: 0.7727
Testing  Accuracy: 0.8615
AUC Score: 0.7562


In [75]:
# unpacking the confusion matrix
logreg_tn, \
logreg_fp, \
logreg_fn, \
logreg_tp = confusion_matrix(y_true = y_test, y_pred = logreg_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {logreg_tn}
False Positives: {logreg_fp}
False Negatives: {logreg_fn}
True Positives : {logreg_tp}
""")


True Negatives : 27
False Positives: 23
False Negatives: 4
True Positives : 141



In [76]:
# comparing results
print(f"""
Model         AUC Score      TN, FP, FN, TP
-----         ---------      --------------
Logistic      {logreg_auc_score}         {logreg_tn, logreg_fp, logreg_fn, logreg_tp}
""")
# Full Tree     {full_tree_auc_score}           {full_tree_tn, full_tree_fp, full_tree_fn, full_tree_tp}
# Pruned Tree   {pruned_tree_auc_score}         {pruned_tree_tn, pruned_tree_fp, pruned_tree_fn, pruned_tree_tp}


# creating a dictionary for model results
model_performance = {
    
    'Model Name'    : ['Logistic'],
           
    'AUC Score' : [logreg_auc_score],
    
    'Training Accuracy' : [logreg_train_score],
           
    'Testing Accuracy'  : [logreg_test_score],

    'Confusion Matrix'  : [(logreg_tn, logreg_fp, logreg_fn, logreg_tp)]}


# converting model_performance into a DataFrame
model_performance = pd.DataFrame(model_performance)



Model         AUC Score      TN, FP, FN, TP
-----         ---------      --------------
Logistic      0.7562         (27, 23, 4, 141)



### Tuning 2-AUC Score: 0.7497

In [77]:
# # train/test split with the full model
# GOT_data   =  GOT.loc[ : , candidate_dict['logit_sig_2']]
# GOT_target =  GOT.loc[ : , 'isAlive']


# # This is the exact code we were using before
# x_train, x_test, y_train, y_test = train_test_split(
#             GOT_data,
#             GOT_target,
#             test_size    = 0.10,
#             random_state = 219,
#             stratify     = GOT_target)


# # INSTANTIATING a logistic regression model
# logreg = LogisticRegression(solver = 'lbfgs',
#                             C = 4.3,
#                             warm_start=True,
#                             random_state = 219,
#                             max_iter = 3000)


# # FITTING the training data
# logreg_fit = logreg.fit(x_train, y_train)

# # PREDICTING based on the testing set
# logreg_pred = logreg_fit.predict(x_test)

# # SCORING the results
# print('Training Accuracy:', logreg_fit.score(x_train, y_train).round(4))
# print('Testing  Accuracy:', logreg_fit.score(x_test, y_test).round(4))

# # area under the roc curve (auc)
# print('AUC Score:', roc_auc_score(y_true  = y_test,
#                     y_score = logreg_pred).round(decimals = 4))


# # saving scoring data for future use
# logreg_train_score = logreg_fit.score(x_train, y_train).round(4) # accuracy
# logreg_test_score  = logreg_fit.score(x_test, y_test).round(4) # accuracy

# # saving AUC score for future use
# logreg_auc_score = roc_auc_score(y_true  = y_test,
#                                  y_score = logreg_pred).round(decimals = 4)

In [78]:
# lr_tuned_cv.best_estimator_

### Tuning 2-AUC Score: 0.7397

In [79]:
# # train/test split with the full model
# GOT_data   =  GOT.loc[ : , candidate_dict['logit_sig_2']]
# GOT_target =  GOT.loc[ : , 'isAlive']


# # This is the exact code we were using before
# x_train, x_test, y_train, y_test = train_test_split(
#             GOT_data,
#             GOT_target,
#             test_size    = 0.10,
#             random_state = 219,
#             stratify     = GOT_target)


# # INSTANTIATING a logistic regression model
# logreg = LogisticRegression(solver = 'newton-cg',
#                             C = 0.4,
#                             warm_start=True,
#                             random_state = 219,
#                             max_iter = 3000)


# # FITTING the training data
# logreg_fit = logreg.fit(x_train, y_train)

# # PREDICTING based on the testing set
# logreg_pred = logreg_fit.predict(x_test)

# # SCORING the results
# print('Training Accuracy:', logreg_fit.score(x_train, y_train).round(4))
# print('Testing  Accuracy:', logreg_fit.score(x_test, y_test).round(4))

# # area under the roc curve (auc)
# print('AUC Score:', roc_auc_score(y_true  = y_test,
#                     y_score = logreg_pred).round(decimals = 4))


# # saving scoring data for future use
# logreg_train_score = logreg_fit.score(x_train, y_train).round(4) # accuracy
# logreg_test_score  = logreg_fit.score(x_test, y_test).round(4) # accuracy

# # saving AUC score for future use
# logreg_auc_score = roc_auc_score(y_true  = y_test,
#                                  y_score = logreg_pred).round(decimals = 4)

# Cart model

In [80]:
########################################
# plot_feature_importances
########################################
def plot_feature_importances(model, train, export = False):
    """
    Plots the importance of features from a CART model.
    
    PARAMETERS
    ----------
    model  : CART model
    train  : explanatory variable training data
    export : whether or not to export as a .png image, default False
    """
    
    # declaring the number
    n_features = x_train.shape[1]
    
    # setting plot window
    fig, ax = plt.subplots(figsize=(12,9))
    
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), train.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    
    if export == True:
        plt.savefig('Tree_Leaf_50_Feature_Importance.png')

# Full Tree

In [81]:
# INSTANTIATING a classification tree object
full_tree = DecisionTreeClassifier()


# FITTING the training data
full_tree_fit = full_tree.fit(x_train, y_train)


# PREDICTING on new data
full_tree_pred = full_tree_fit.predict(x_test)


# SCORING the model
print('Full Tree Training ACCURACY:', full_tree_fit.score(x_train,
                                                     y_train).round(4))

print('Full Tree Testing ACCURACY :', full_tree_fit.score(x_test,
                                                     y_test).round(4))

print('Full Tree AUC Score:', roc_auc_score(y_true  = y_test,
                                            y_score = full_tree_pred).round(4))


# saving scoring data for future use
full_tree_train_score = full_tree_fit.score(x_train, y_train).round(4) # accuracy
full_tree_test_score  = full_tree_fit.score(x_test, y_test).round(4)   # accuracy


# saving AUC
full_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                      y_score = full_tree_pred).round(4) # auc

Full Tree Training ACCURACY: 1.0
Full Tree Testing ACCURACY : 0.7333
Full Tree AUC Score: 0.6766


In [82]:
# unpacking the confusion matrix
full_tree_tn, \
full_tree_fp, \
full_tree_fn, \
full_tree_tp = confusion_matrix(y_true = y_test, y_pred = full_tree_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {full_tree_tn}
False Positives: {full_tree_fp}
False Negatives: {full_tree_fn}
True Positives : {full_tree_tp}
""")


True Negatives : 28
False Positives: 22
False Negatives: 30
True Positives : 115



In [83]:
# declaring model performance objects
full_tree_train_score = full_tree_fit.score(x_train, y_train).round(4)
full_tree_test_score  = full_tree_fit.score(x_test, y_test).round(4)
full_tree_auc_score       = roc_auc_score(y_true  = y_test,
                             y_score = full_tree_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'         : 'Full Tree',
                           'Training Accuracy'  : full_tree_train_score,
                           'Testing Accuracy'   : full_tree_test_score,
                           'AUC Score'          : full_tree_auc_score,
                           'Confusion Matrix'   : (full_tree_tn,
                                                   full_tree_fp,
                                                   full_tree_fn,
                                                   full_tree_tp)},
                          ignore_index = True)

# checking the results
model_performance

,Model Name,AUC Score,Training Accuracy,Testing Accuracy,Confusion Matrix
0,Logistic,0.7562,0.7727,0.8615,"(27, 23, 4, 141)"
1,Full Tree,0.6766,1.0000,0.7333,"(28, 22, 30, 115)"


# Pruned Tree

## General settings

In [84]:
# INSTANTIATING a classification tree object
tree_pruned = DecisionTreeClassifier(max_depth=6.9,
                    min_samples_leaf=24,
                    random_state = 219)


# FITTING the training data
tree_pruned_fit = tree_pruned.fit(x_train, y_train)


# PREDICTING on new data
tree_pruned_predict = tree_pruned_fit.predict(x_test)


# SCORING the model
print('Training ACCURACY:', tree_pruned_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', tree_pruned_fit.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_pruned_predict).round(4))


# saving scoring data for future use
pruned_tree_train_score = tree_pruned_fit.score(x_train, y_train).round(4) # accuracy
pruned_tree_test_score  = tree_pruned_fit.score(x_test, y_test).round(4) # accuracy


# saving auc score
pruned_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                        y_score = tree_pruned_predict).round(4) # auc

Training ACCURACY: 0.8058
Testing  ACCURACY: 0.841
AUC Score        : 0.7359


## Tuning
For the tuning, I decided to go with values that scikit learn suggested as good ranges

In [85]:
# # declaring a hyperparameter space
# criterion_range=["gini","entropy"]
# max_depth_range= np.arange(0.1,8,0.1)
# splitter_range= ["random"]
# min_samples_split_range=np.arange(2,7,1)

# # depth range and leaf range

# # creating a hyperparameter grid
# param_grid = {'criterion'          : criterion_range,
#              'max_depth'          : max_depth_range,
#              'splitter'          : splitter_range,
#              'min_samples_split':min_samples_split_range}
           



# # INSTANTIATING the model object without hyperparameters
# lr_tuned = DecisionTreeClassifier(random_state = 219)



# # RandomizedSearchCV object
# lr_tuned_cv = RandomizedSearchCV(estimator           = lr_tuned,   # the model object
#                                  param_distributions = param_grid, # parameters to tune
#                                  cv                  = 3,          # how many folds in cross-validation
#                                  n_iter              = 250,        # number of combinations of hyperparameters to try
#                                  random_state        = 219,        # starting point for random sequence
#                                  scoring = make_scorer(
#                                            roc_auc_score,
#                                            needs_threshold = False))



# # FITTING to the FULL DATASET (due to cross-validation)
# lr_tuned_cv.fit(GOT_data_2, GOT_target)



# # printing the optimal parameters and best score

# print("Tuned Parameters  :", lr_tuned_cv.best_params_)
# print("Tuned CV AUC      :", lr_tuned_cv.best_score_.round(4))

## Best try

In [86]:
# INSTANTIATING a classification tree object
tree_pruned = DecisionTreeClassifier(max_depth=6.1,
                                     splitter='random',
                           min_samples_split=5,         
                    criterion='gini',
                    random_state = 219)


# FITTING the training data
tree_pruned_fit = tree_pruned.fit(x_train, y_train)


# PREDICTING on new data
tree_pruned_predict = tree_pruned_fit.predict(x_test)


# SCORING the model
print('Training ACCURACY:', tree_pruned_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', tree_pruned_fit.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_pruned_predict).round(4))


# saving scoring data for future use
pruned_tree_train_score = tree_pruned_fit.score(x_train, y_train).round(4) # accuracy
pruned_tree_test_score  = tree_pruned_fit.score(x_test, y_test).round(4) # accuracy


# saving auc score
pruned_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                        y_score = tree_pruned_predict).round(4) # auc

Training ACCURACY: 0.803
Testing  ACCURACY: 0.8718
AUC Score        : 0.7631


In [87]:
# unpacking the confusion matrix
pruned_tree_tn, \
pruned_tree_fp, \
pruned_tree_fn, \
pruned_tree_tp = confusion_matrix(y_true = y_test, y_pred = tree_pruned_predict).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {pruned_tree_tn}
False Positives: {pruned_tree_fp}
False Negatives: {pruned_tree_fn}
True Positives : {pruned_tree_tp}
""")


True Negatives : 27
False Positives: 23
False Negatives: 2
True Positives : 143



In [88]:
# declaring model performance objects
pruned_tree_train_score = tree_pruned_fit.score(x_train, y_train).round(4)
pruned_tree_test_score  = tree_pruned_fit.score(x_test, y_test).round(4)
pruned_tree_auc_score       = roc_auc_score(y_true  = y_test,
                             y_score = tree_pruned_predict).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'         : 'Pruned Tree',
                           'Training Accuracy'  : pruned_tree_train_score,
                           'Testing Accuracy'   : pruned_tree_test_score,
                           'AUC Score'          : pruned_tree_auc_score,
                           'Confusion Matrix'   : (pruned_tree_tn,
                                                   pruned_tree_fp,
                                                   pruned_tree_fn,
                                                   pruned_tree_tp)},
                          ignore_index = True)

# checking the results
model_performance

,Model Name,AUC Score,Training Accuracy,Testing Accuracy,Confusion Matrix
0,Logistic,0.7562,0.7727,0.8615,"(27, 23, 4, 141)"
1,Full Tree,0.6766,1.0000,0.7333,"(28, 22, 30, 115)"
2,Pruned Tree,0.7631,0.8030,0.8718,"(27, 23, 2, 143)"


In [89]:
# # INSTANTIATING a classification tree object
# tree_pruned = DecisionTreeClassifier(max_depth=6.8,
#                     min_samples_leaf=3,
#                     splitter= 'random',
#                     criterion= 'gini',
#                     random_state = 219)


# # FITTING the training data
# tree_pruned_fit = tree_pruned.fit(x_train, y_train)


# # PREDICTING on new data
# tree_pruned_predict = tree_pruned_fit.predict(x_test)


# # SCORING the model
# print('Training ACCURACY:', tree_pruned_fit.score(x_train, y_train).round(4))
# print('Testing  ACCURACY:', tree_pruned_fit.score(x_test, y_test).round(4))
# print('AUC Score        :', roc_auc_score(y_true  = y_test,
#                                           y_score = tree_pruned_predict).round(4))


# # saving scoring data for future use
# pruned_tree_train_score = tree_pruned_fit.score(x_train, y_train).round(4) # accuracy
# pruned_tree_test_score  = tree_pruned_fit.score(x_test, y_test).round(4) # accuracy


# # saving auc score
# pruned_tree_auc_score   = roc_auc_score(y_true  = y_test,
#                                         y_score = tree_pruned_predict).round(4) # auc

In [90]:
########################################
# plot_feature_importances
########################################
def plot_feature_importances(model, train, export = False):
    """
    Plots the importance of features from a CART model.
    
    PARAMETERS
    ----------
    model  : CART model
    train  : explanatory variable training data
    export : whether or not to export as a .png image, default False
    """
    
    # declaring the number
    n_features = train.shape[1]
    
    # setting plot window
    fig, ax = plt.subplots(figsize=(12,9))
    
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), train.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    
    if export == True:
        plt.savefig('./analysis_images/Feature_Importance.png')

In [91]:
# # train/test split with the logit_sig variables
# GOT_data_2   =  GOT.loc[:,candidate_dict['logit_full']]
# GOT_target =  GOT.loc[:,'isAlive']


# # train/test split
# x_train, x_test, y_train, y_test = train_test_split(
#             GOT_data_2,
#             GOT_target,
#             random_state = 219,
#             test_size    = 0.10,
#             stratify     = GOT_target)

# Random Forest

## Default

In [92]:
# INSTANTIATING a random forest model with default values
rf_default = RandomForestClassifier(n_estimators     = 100,
                                    criterion        = 'gini',
                                    max_depth        = 4,
                                    min_samples_leaf = 1,
                                    bootstrap        = True,
                                    warm_start       = False,
                                    random_state     = 219)

In [93]:
# FITTING the training data
rf_default_fit = rf_default.fit(x_train, y_train)


# PREDICTING based on the testing set
rf_default_fit_pred = rf_default_fit.predict(x_test)


# SCORING the results
print('Training ACCURACY:', rf_default_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', rf_default_fit.score(x_test, y_test).round(4))


# saving AUC score
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = rf_default_fit_pred).round(4))

Training ACCURACY: 0.7973
Testing  ACCURACY: 0.8462
AUC Score        : 0.7


## Tuning
For the tuning, I decided to go with values that scikit learn suggested as good ranges

In [94]:
# # FITTING the training data
# rf_default_fit = rf_default.fit(x_train, y_train)


# # PREDICTING based on the testing set
# rf_default_fit_pred = rf_default_fit.predict(x_test)


# # declaring a hyperparameter space
# estimator_range  = np.arange(100, 1100, 250)
# max_depth_range= np. arange(3,6,0.1)
# leaf_range       = np.arange(10, 35, 5)
# criterion_range  = ['gini', 'entropy']
# bootstrap_range  = [True, False]
# warm_start_range = [True, False]


# # creating a hyperparameter grid
# param_grid = {'n_estimators'     : estimator_range,
#               'max_depth'         :max_depth_range,
#               'min_samples_leaf'  : leaf_range,
#               'criterion'        : criterion_range,
#               'bootstrap'        : bootstrap_range,
#               'warm_start'       : warm_start_range}


# # INSTANTIATING the model object without hyperparameters
# forest_grid = RandomForestClassifier(random_state = 219)


# # GridSearchCV object
# forest_cv = RandomizedSearchCV(estimator           = forest_grid,
#                                param_distributions = param_grid,
#                                cv         = 3,
#                                n_iter     = 1000,
#                                scoring    = make_scorer(roc_auc_score,
#                                             needs_threshold = False))


# # FITTING to the FULL DATASET (due to cross-validation)
# forest_cv.fit(GOT_data_2, GOT_target)


# # PREDICT step is not needed


# # printing the optimal parameters and best score
# print("Tuned Parameters  :", forest_cv.best_params_)
# print("Tuned Training AUC:", forest_cv.best_score_.round(4))

In [95]:
#forest_cv.best_estimator_

In [96]:
# building a model based on hyperparameter tuning results

# INSTANTIATING with best_estimator
forest_tuned = RandomForestClassifier(criterion= 'gini',
                                     min_samples_leaf=30,
                                       n_estimators=850,
                                      random_state=219, 
                                      warm_start=True,
                                      max_depth=5,
                                      bootstrap= True
                                     )


# FITTING to the FULL DATASET (due to cross-validation)
forest_tuned_fit = forest_tuned.fit(GOT_data_2, GOT_target)


# PREDICTING based on the testing set
forest_tuned_pred = forest_tuned_fit.predict(x_test)


# SCORING the results
print('Forest Tuned Training ACCURACY:', forest_tuned.score(x_train, y_train).round(4))
print('Forest Tuned Testing  ACCURACY:', forest_tuned.score(x_test, y_test).round(4))
print('Forest Tuned AUC Score        :', roc_auc_score(y_true  = y_test,
                                                       y_score = forest_tuned_pred).round(4))


# saving scoring data for future use
forest_tuned_train_score = forest_tuned.score(x_train, y_train).round(4) # accuracy
forest_tuned_test_score  = forest_tuned.score(x_test, y_test).round(4)   # accuracy


# saving the AUC score?
forest_tuned_auc = roc_auc_score(y_true  = y_test,
                                 y_score = forest_tuned_pred).round(4) # auc

Forest Tuned Training ACCURACY: 0.7973
Forest Tuned Testing  ACCURACY: 0.8718
Forest Tuned AUC Score        : 0.75


# Gradient Boosted Machines
For the tuning, I decided to go with values that scikit learn suggested as good ranges

In [97]:
# # declaring a hyperparameter space
# learn_range        = np.arange(0.1, 0.5, 0.01)
# estimator_range    = np.arange(40, 60, 1)
# depth_range        = np.arange(2, 8, 2)
# warm_start_range   = [True, False]
# loss_range         = ['deviance', 'exponential']
# criterion_range    =  ['friedman_mse','mse']

# # creating a hyperparameter grid
# param_grid = {'learning_rate' : learn_range,
#               'max_depth'     : depth_range,
#               'n_estimators'  : estimator_range,
#               'warm_start'    : warm_start_range,
#              'loss'            : loss_range,
#              'criterion'        : criterion_range}


# # INSTANTIATING the model object without hyperparameters
# full_gbm_grid = GradientBoostingClassifier(random_state = 219)


# # GridSearchCV object
# full_gbm_cv = RandomizedSearchCV(estimator     = full_gbm_grid,
#                            param_distributions = param_grid,
#                            cv                  = 3,
#                            n_iter              = 900,
#                            random_state        = 219,
#                            scoring             = make_scorer(roc_auc_score,
#                                                  needs_threshold = False))


# # FITTING to the FULL DATASET (due to cross-validation)
# full_gbm_cv.fit(GOT_data_2, GOT_target)


# # PREDICT step is not needed


# # printing the optimal parameters and best score
# print("Tuned Parameters  :", full_gbm_cv.best_params_)
# print("Tuned Training AUC:", full_gbm_cv.best_score_.round(4))

## Full dataset after Tuning

In [98]:
GOT_data_2= GOT.loc[:, candidate_dict['logit_full']]

# This is the exact code we were using before
x_train, x_test, y_train, y_test = train_test_split(
            GOT_data_2,
            GOT_target,
            test_size    = 0.10,
            random_state = 219,
            stratify     = GOT_target)

In [99]:
# # INSTANTIATING the model object without hyperparameters
# full_gbm_default = GradientBoostingClassifier(loss          = 'deviance',
#                                               learning_rate = 0.1,
#                                               n_estimators  = 100,
#                                               criterion     = 'friedman_mse',
#                                               max_depth     = 3,
#                                               warm_start    = False,
#                                               random_state  = 219)


# # FIT step is needed as we are not using .best_estimator
# full_gbm_default_fit = full_gbm_default.fit(x_train, y_train)


# # PREDICTING based on the testing set
# full_gbm_default_pred = full_gbm_default_fit.predict(x_test)


# # SCORING the results
# print('Training ACCURACY:', full_gbm_default_fit.score(x_train, y_train).round(4))
# print('Testing ACCURACY :', full_gbm_default_fit.score(x_test, y_test).round(4))
# print('AUC Score        :', roc_auc_score(y_true  = y_test,
#                                           y_score = full_gbm_default_pred).round(4))

In [100]:
#Best score
full_gbm = GradientBoostingClassifier(loss          = 'deviance',
                                              learning_rate = 0.41, 
                                              n_estimators  = 68,
                                              criterion     = 'friedman_mse',
                                              max_depth     = 1,
                                              warm_start    = True,
                                              random_state  = 219)


# FIT step is needed as we are not using .best_estimator
full_gbm_fit = full_gbm.fit(x_train, y_train)


# PREDICTING based on the testing set
full_gbm_pred = full_gbm_fit.predict(x_test)


# SCORING the results
print('Training ACCURACY:', full_gbm_fit.score(x_train, y_train).round(4))
print('Testing ACCURACY :', full_gbm_fit.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = full_gbm_pred).round(4))



Training ACCURACY: 0.8013
Testing ACCURACY : 0.8923
AUC Score        : 0.8228


In [101]:
#second best score
# Training ACCURACY: 0.7961
# Testing ACCURACY : 0.8872
# AUC Score        : 0.8128

# full_gbm = GradientBoostingClassifier(loss          = 'exponential',
#                                               learning_rate = 0.28,
#                                               n_estimators  = 62,
#                                               criterion     = 'friedman_mse',
#                                               max_depth     = 1,
#                                               warm_start    = True,
#                                               random_state  = 219)


# # FIT step is needed as we are not using .best_estimator
# full_gbm_fit = full_gbm.fit(x_train, y_train)


# # PREDICTING based on the testing set
# full_gbm_pred = full_gbm_fit.predict(x_test)


# # SCORING the results
# print('Training ACCURACY:', full_gbm_fit.score(x_train, y_train).round(4))
# print('Testing ACCURACY :', full_gbm_fit.score(x_test, y_test).round(4))
# print('AUC Score        :', roc_auc_score(y_true  = y_test,
#                                           y_score = full_gbm_pred).round(4))



In [102]:
#Third best score

# Training ACCURACY: 0.8178
# Testing ACCURACY : 0.8821
# AUC Score        : 0.7962

# full_gbm = GradientBoostingClassifier(loss          = 'deviance',
#                                               learning_rate = 0.2,
#                                               n_estimators  = 45,
#                                               criterion     = 'friedman_mse',
#                                               max_depth     = 2,
#                                               warm_start    = True,
#                                               random_state  = 219)


# # FIT step is needed as we are not using .best_estimator
# full_gbm_fit = full_gbm.fit(x_train, y_train)


# # PREDICTING based on the testing set
# full_gbm_pred = full_gbm_fit.predict(x_test)


# # SCORING the results
# print('Training ACCURACY:', full_gbm_fit.score(x_train, y_train).round(4))
# print('Testing ACCURACY :', full_gbm_fit.score(x_test, y_test).round(4))
# print('AUC Score        :', roc_auc_score(y_true  = y_test,
#                                           y_score = full_gbm_pred).round(4))



In [103]:
# unpacking the confusion matrix
gbm_tn, \
gbm_fp, \
gbm_fn, \
gbm_tp = confusion_matrix(y_true = y_test, y_pred = full_gbm_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_tn}
False Positives: {gbm_fp}
False Negatives: {gbm_fn}
True Positives : {gbm_tp}
""")


True Negatives : 34
False Positives: 16
False Negatives: 5
True Positives : 140



In [104]:
# declaring model performance objects
gbm_train_acc = full_gbm_fit.score(x_train, y_train).round(4)
gbm_test_acc  = full_gbm_fit.score(x_test, y_test).round(4)
gbm_auc       = roc_auc_score(y_true  = y_test,
                              y_score = full_gbm_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'       : 'GBM (Full) [Final]',
                          'Training Accuracy' : gbm_train_acc,
                          'Testing Accuracy'  : gbm_test_acc,
                          'AUC Score'         : gbm_auc,
                          'Confusion Matrix'  : (gbm_tn,
                                                 gbm_fp,
                                                 gbm_fn,
                                                 gbm_tp)},
                          ignore_index = True)


# checking the results
model_performance

,Model Name,AUC Score,Training Accuracy,Testing Accuracy,Confusion Matrix
0,Logistic,0.7562,0.7727,0.8615,"(27, 23, 4, 141)"
1,Full Tree,0.6766,1.0000,0.7333,"(28, 22, 30, 115)"
2,Pruned Tree,0.7631,0.8030,0.8718,"(27, 23, 2, 143)"
3,GBM (Full) [Final],0.8228,0.8013,0.8923,"(34, 16, 5, 140)"


## Reduced full

In [105]:
# explanatory sets from last session

# creating a dictionary to store candidate models

candidate_dict = {

 # full model
 'logit_full'   : ['Order' , 'book1_A_Game_Of_Thrones' , 'book2_A_Clash_Of_Kings' , \
                   'book3_A_Storm_Of_Swords' , 'book4_A_Feast_For_Crows' , 'book5_A_Dance_with_Dragons' ,\
                   'isMarried' , 'isNoble' , 'numDeadRelations' , 'popularity' , 'mother_unknown' ,\
                   'father_unknown' , 'house_unknown' , 'culture_unknown' , 'title_unknown' , \
                   'heir_unknown' , 'spouse_unknown' , 'age_unknown' , 'isAliveMother_unknown' , \
                   'isAliveFather_unknown' , 'isAliveHeir_unknown' , 'isAliveSpouse_unknown'], 
    # full model
 'logit_full_remove'   : ['Order' , 'book1_A_Game_Of_Thrones' , 'book2_A_Clash_Of_Kings' , \
                  'book3_A_Storm_Of_Swords','book4_A_Feast_For_Crows' , 'book5_A_Dance_with_Dragons' ,\
                    'numDeadRelations' , 'popularity' ,'culture_unknown' ,'isMarried', 'title_unknown' , \
                   'spouse_unknown' , 'age_unknown' , 'isAliveSpouse_unknown'], 
    
 # significant variables only (set 1)
 'logit_full_reduced'    : ['Order' , 'book1_A_Game_Of_Thrones', 'book4_A_Feast_For_Crows',\
                          'mother_unknown','age_unknown','alive_books_longest'],
    
 # significant variables only (set 1)
 'logit_sig'    : ['Order' , 'book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings',\
                   'book4_A_Feast_For_Crows','numDeadRelations', 'mother_unknown','age_unknown'],
    
  
 # significant variables only (set 2)
 'logit_sig_2'  : ['book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings', 'book4_A_Feast_For_Crows',\
                   'mother_unknown', 'age_unknown', 'popularity_survive', 'Order'],

                   
    # mostly significant variables (set 3)
 'logit_unsig_3'  : ['Order', 'book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings',\
                   'book4_A_Feast_For_Crows', 'numDeadRelations', 'mother_unknown',\
                     'familiarity_father_2','age_unknown'],     
    
    # significant variables only (without 'mother_unknown')
     'logit_sig_2_1'  : ['book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings', 'book4_A_Feast_For_Crows',\
                    'age_unknown', 'popularity_survive', 'Order'],
    # significant variables only (without 'mother_unknown')
     'logit_sig_2_2'  : ['Order', 'book1_A_Game_Of_Thrones', 'book2_A_Clash_Of_Kings',\
                    'book4_A_Feast_For_Crows', 'numDeadRelations', 'mother_unknown', 'age_unknown'],
    
    # significant variables only (set 2)
 'logit_sig_4'  : ['book4_A_Feast_For_Crows', 'age_unknown', 'father_unknown',\
                   'house_unknown', 'title_unknown'],                                   
}

In [106]:
GOT_data_2= GOT.loc[:, candidate_dict['logit_full_remove']]

# This is the exact code we were using before
x_train, x_test, y_train, y_test = train_test_split(
            GOT_data_2,
            GOT_target,
            test_size    = 0.10,
            random_state = 219,
            stratify     = GOT_target)
full_gbm_remove = GradientBoostingClassifier(loss          = 'deviance',
                                              learning_rate = 0.45,
                                              n_estimators  = 48,
                                              criterion     = 'friedman_mse',
                                              max_depth     = 2,
                                              warm_start    = True,
                                              random_state  = 219)


# FIT step is needed as we are not using .best_estimator
full_gbm_remove_fit = full_gbm_remove.fit(x_train, y_train)


# PREDICTING based on the testing set
full_gbm_remove_pred = full_gbm_remove_fit.predict(x_test)


# SCORING the results
print('Training ACCURACY:', full_gbm_remove_fit.score(x_train, y_train).round(4))
print('Testing ACCURACY :', full_gbm_remove_fit.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = full_gbm_remove_pred).round(4))

Training ACCURACY: 0.8395
Testing ACCURACY : 0.8615
AUC Score        : 0.789


In [107]:
# unpacking the confusion matrix
gbm_remove_tn, \
gbm_remove_fp, \
gbm_remove_fn, \
gbm_remove_tp = confusion_matrix(y_true = y_test, y_pred = full_gbm_remove_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_remove_tn}
False Positives: {gbm_remove_fp}
False Negatives: {gbm_remove_fn}
True Positives : {gbm_remove_tp}
""")


True Negatives : 32
False Positives: 18
False Negatives: 9
True Positives : 136



In [108]:
# declaring model performance objects
gbm_remove_train_acc = full_gbm_remove_fit.score(x_train, y_train).round(4)
gbm_remove_test_acc  = full_gbm_remove_fit.score(x_test, y_test).round(4)
gbm_remove_auc       = roc_auc_score(y_true  = y_test,
                              y_score = full_gbm_remove_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'       : 'GBM (Full_removed)',
                          'Training Accuracy' : gbm_remove_train_acc,
                          'Testing Accuracy'  : gbm_remove_test_acc,
                          'AUC Score'         : gbm_remove_auc,
                          'Confusion Matrix'  : (gbm_remove_tn,
                                                 gbm_remove_fp,
                                                 gbm_remove_fn,
                                                 gbm_remove_tp)},
                          ignore_index = True)


# checking the results
model_performance

,Model Name,AUC Score,Training Accuracy,Testing Accuracy,Confusion Matrix
0,Logistic,0.7562,0.7727,0.8615,"(27, 23, 4, 141)"
1,Full Tree,0.6766,1.0000,0.7333,"(28, 22, 30, 115)"
2,Pruned Tree,0.7631,0.8030,0.8718,"(27, 23, 2, 143)"
3,GBM (Full) [Final],0.8228,0.8013,0.8923,"(34, 16, 5, 140)"
4,GBM (Full_removed),0.7890,0.8395,0.8615,"(32, 18, 9, 136)"


## Sig_2_2_set

In [109]:
GOT_data_2= GOT.loc[:, candidate_dict['logit_sig_2_2']]

# This is the exact code we were using before
x_train, x_test, y_train, y_test = train_test_split(
            GOT_data_2,
            GOT_target,
            test_size    = 0.10,
            random_state = 219,
            stratify     = GOT_target)
full_gbm_sig = GradientBoostingClassifier(loss          = 'deviance',
                                              learning_rate = 0.22,
                                              n_estimators  = 50,
                                              criterion     = 'friedman_mse',
                                              max_depth     = 4,
                                              warm_start    = True,
                                              random_state  = 219)


# FIT step is needed as we are not using .best_estimator
full_gbm_sig_fit = full_gbm_sig.fit(x_train, y_train)


# PREDICTING based on the testing set
full_gbm_sig_pred = full_gbm_sig_fit.predict(x_test)


# SCORING the results
print('Training ACCURACY:', full_gbm_sig_fit.score(x_train, y_train).round(4))
print('Testing ACCURACY :', full_gbm_sig_fit.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = full_gbm_sig_pred).round(4))

Training ACCURACY: 0.8698
Testing ACCURACY : 0.8615
AUC Score        : 0.789


In [110]:
# unpacking the confusion matrix
gbm_sig_tn, \
gbm_sig_fp, \
gbm_sig_fn, \
gbm_sig_tp = confusion_matrix(y_true = y_test, y_pred = full_gbm_sig_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_sig_tn}
False Positives: {gbm_sig_fp}
False Negatives: {gbm_sig_fn}
True Positives : {gbm_sig_tp}""")


True Negatives : 32
False Positives: 18
False Negatives: 9
True Positives : 136


In [111]:
# declaring model performance objects
gbm_sig_train_acc = full_gbm_sig_fit.score(x_train, y_train).round(4)
gbm_sig_test_acc  = full_gbm_sig_fit.score(x_test, y_test).round(4)
gbm_sig_auc       = roc_auc_score(y_true  = y_test,
                              y_score = full_gbm_sig_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'       : 'GBM (sig)',
                          'Training Accuracy' : gbm_sig_train_acc,
                          'Testing Accuracy'  : gbm_sig_test_acc,
                          'AUC Score'         : gbm_sig_auc,
                          'Confusion Matrix'  : (gbm_sig_tn,
                                                 gbm_sig_fp,
                                                 gbm_sig_fn,
                                                 gbm_sig_tp)},
                          ignore_index = True)


# checking the results
model_performance

,Model Name,AUC Score,Training Accuracy,Testing Accuracy,Confusion Matrix
0,Logistic,0.7562,0.7727,0.8615,"(27, 23, 4, 141)"
1,Full Tree,0.6766,1.0000,0.7333,"(28, 22, 30, 115)"
2,Pruned Tree,0.7631,0.8030,0.8718,"(27, 23, 2, 143)"
3,GBM (Full) [Final],0.8228,0.8013,0.8923,"(34, 16, 5, 140)"
4,GBM (Full_removed),0.7890,0.8395,0.8615,"(32, 18, 9, 136)"
5,GBM (sig),0.7890,0.8698,0.8615,"(32, 18, 9, 136)"


In [112]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 6.78459906578064 seconds ---
